# 1. Import Library

In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv
from pathlib import Path

import re

from sentence_transformers import SentenceTransformer

import faiss
from openai import OpenAI

from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [7]:
pd.set_option('display.max_colwidth', 500)

# 2. Preparing Data

In [8]:
full_df = pd.read_csv('../artikel/raw_health_articles.csv')
df = full_df[['content', 'source']]
df.head()

,content,source
0,"Ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. Jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. Banyak penyakit ginjal apa saja yang berkembang tanpa gejala pada tahap awal, sehingga penting menjaga kesehatan ginjal sejak dini.\nBerikut ini adalah 5 cara merawat ginjal yang sederhana namun efektif untuk mencegah gejala ginjal bermasalah:\n1. Minum Air Putih yang Cukup\nAsupan cairan yang cukup mem...",Biofarma
1,"Menjaga kesehatan mata sangatlah penting karena mata berperan besar dalam hampir semua aktivitas sehari-hari, mulai dari belajar, bekerja, hingga berinteraksi dengan lingkungan sekitar. Jika kesehatan mata terabaikan, berbagai masalah bisa muncul, mulai dari iritasi ringan, infeksi, hingga penyakit serius yang berisiko menyebabkan kebutaan. Untuk itu, menjaga mata dapat dilakukan melalui pemeriksaan rutin ke dokter dan juga dengan membiasakan pola hidup sehat yang mendukung fungsi penglihata...",Biofarma
2,"Penyebab Campak\nCampak adalah penyakit menular yang disebabkan oleh virus Morbili dan dapat menyebar dengan sangat cepat dari satu orang ke orang lain. Mekanisme utama penularannya adalah melalui percikan air liur (droplet) yang keluar dari mulut atau hidung penderita saat batuk, bersin, atau berbicara. Selain itu, seseorang juga dapat tertular jika menyentuh benda yang sudah terkontaminasi virus lalu menyentuh hidung, mulut, atau mata. Berikut adalah sepuluh faktor penyebab yang paling ber...",Biofarma
3,"Apa itu campak?\nCampak atau rubella adalah penyakit menular yang disebabkan oleh virus dari keluarga Paramyxoviridae dengan genus Morbillivirus. Penyakit ini menyerang saluran pernapasan terlebih dahulu, lalu menimbulkan gejala sistemik seperti demam, batuk, pilek, sakit tenggorokan, mata merah, hingga ruam menyeluruh di kulit. Campak termasuk penyakit yang sangat menular, bisa berpindah dari orang ke orang hanya melalui udara, percikan air liur, atau benda yang terkontaminasi virus.\nApa s...",Biofarma
4,"Apa itu Kesehatan Gigi dan Mulut?\nKesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk ke dalam tubuh, sekaligus organ yang berperan penting dalam proses pencernaan. Saat makanan masuk, sebagian akan tertinggal dalam mulut. Jika tidak dibersihkan, sisa tersebut bisa menjadi tempat berkembangnya bakteri. Oleh karena itu, kesehatan mulut tidak...",Biofarma


# 3. Convert to Dictionary

In [23]:
df_dict = df.to_dict()

# 4. Text Cleaning

In [25]:
def clean_text(text):
    # Remove the specific UI noise at the end of the text
    text = text.replace("Bagikan:\nDownload", "")
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [28]:
docs = []
for index, content in df_dict['content'].items():
    cleaned_content = clean_text(content)
    # We create a Document object. 
    # We store the 'index' in metadata so we can trace the source later.
    doc = Document(
        page_content=cleaned_content, 
        metadata={"source_id": index}
    )
    docs.append(doc)

print(f"Loaded {len(docs)} documents.")

Loaded 420 documents.


# 5. Chunking

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""] # Priorities for splitting
)

In [30]:
splits = text_splitter.split_documents(docs)
print(f"Created {len(splits)} chunks from {len(docs)} original documents.")

Created 2712 chunks from 420 original documents.


# 6. Embedding

In [33]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [34]:
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embedding_model
)

# 7. Retrieve Document

In [35]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # Retrieve the top 5 most relevant chunks
)

In [36]:
# Test retrieval
query = "Bagaimana cara menjaga kesehatan ginjal?"
retrieved_docs = retriever.invoke(query)

In [37]:
print(f"Query: {query}")
for i, doc in enumerate(retrieved_docs):
    print(f"\n[Result {i+1}] (Source ID: {doc.metadata['source_id']})")
    print(doc.page_content[:200] + "...")

Query: Bagaimana cara menjaga kesehatan ginjal?

[Result 1] (Source ID: 223)
. Beberapa orang yang didiagnosis memiliki kadar kreatinin normal, mungkin saja memiliki penyakit ginjal. Begitu juga sebaliknya, individu dengan kadar kreatinin tinggi belum tentu mengalami penyakit ...

[Result 2] (Source ID: 223)
. Jagalah kesehatan ginjalmu dengan menjaga hidrasi yang cukup, mengendalikan tekanan darah dan kadar gula darah, menjalani gaya hidup sehat, dan melakukan pemeriksaan kesehatan rutin. Jika kamu memil...

[Result 3] (Source ID: 223)
. Tes kreatinin biasanya digunakan untuk memantau kesehatan ginjal dan mendeteksi kemungkinan gangguan ginjal. Apabila setelah pemeriksaan ditemukan adanya gangguan pada organ ginjal, dokter bisa saja...

[Result 4] (Source ID: 37)
Ginjal adalah organ penting yang bekerja tanpa henti untuk menyaring limbah, menjaga keseimbangan elektrolit, dan mengatur tekanan darah. Sayangnya, banyak orang baru menyadari pentingnya fungsi ginja...

[Result 5] (Source I

# 8. Complete Retrieval Augmented Generation (RAG)

In [45]:
# Check Gemini API
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY:
    print("API Key ditemukan!")
else:
    print("API Key masih None. Periksa path file .env")

API Key ditemukan!


In [54]:
client = OpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

def generate_augmented_response(query):
  search_results = retriever.invoke(query)
  context = "\n\n".join([doc.page_content for doc in search_results])

  prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
  response = client.chat.completions.create(
      model="gemini-2.5-flash", # Using model gemini 2.5 flash
      messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant."
        },
        {
          "role": "user",
          "content": prompt
        }
      ],
  )

  return response.choices[0].message.content

## 8.1 Factual Queries

In [62]:
query = "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin."
response = generate_augmented_response(query)
print(response)

Berikut adalah zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata, beserta bagian yang mendukung setiap poin dari teks yang diberikan:

1.  **Beta-karoten (yang diubah menjadi Vitamin A)**
    *   **Manfaat untuk Kesehatan Mata:** Sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, mencegah gangguan mata seperti rabun senja dan degenerasi makula. Vitamin A juga berperan dalam menutrisi mata dan menjaga kelembapannya, sehingga mencegah terjadinya mata kering. Kekurangan vitamin A dapat meningkatkan risiko rabun senja.
    *   **Bagian yang Mendukung:**
        *   "Vitamin ini sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan mata seperti rabun senja dan degenerasi makula." (Paragraf pertama)
        *   "Meningkatkan Kesehatan Mata Wortel kaya akan beta-karoten, yang diubah tubuh menjadi vitamin A. Kekurangan vitamin A dapat meningkatkan risiko rabun senja. Se

## 8.2 Conceptual Understanding?

In [ ]:
query = "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?"
response = generate_augmented_response(query)
print(response)

Penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal karena:

1.  **Garam berlebih meningkatkan tekanan darah:** Asupan garam berlebih dapat meningkatkan tekanan darah.
2.  **Tekanan darah tinggi merusak ginjal:** Tekanan darah tinggi merupakan salah satu faktor utama penyebab penyakit ginjal kronis. Kondisi ini dapat merusak pembuluh darah di ginjal.
3.  **Menghambat aliran darah dan fungsi ginjal:** Jika tekanan darah tinggi dibiarkan, dapat menghambat aliran darah ke ginjal dan menurunkan fungsinya.

Dengan membatasi konsumsi garam, tekanan darah dapat dikendalikan, sehingga membantu menjaga aliran darah ke ginjal tetap lancar dan melindungi fungsi ginjal agar tetap optimal.


## 8.3 Question Requiring Synthesis

In [58]:
query = "Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?"
response = generate_augmented_response(query)
print(response)

Berdasarkan teks yang diberikan, dampak merokok terhadap tubuh, mulai dari kepala hingga organ dalam, adalah sebagai berikut:

**1. Kepala dan Leher:**
*   **Otak:**
    *   **Kerusakan:** Penyempitan pembuluh darah otak akibat nikotin, suplai oksigen ke otak menurun.
    *   **Penyakit/Dampak:** Stroke, mengganggu seluruh sistem tubuh (karena supply oksigen menurun ke organ termasuk otak).
*   **Mulut, Bibir, Tenggorokan (Laring) dan Kerongkongan (Esofagus):**
    *   **Kerusakan:** Tar mengikis selaput lendir, ampas tar menumpuk dan merubah sifat sel normal menjadi sel ganas, panas dari asap.
    *   **Penyakit/Dampak:** Kanker mulut, kanker bibir, kanker tenggorokan (laring), kanker kerongkongan (esofagus).
*   **Indera:**
    *   **Kerusakan/Dampak:** Disebutkan secara umum "Merusak Otak dan Indera," namun teks tidak memerinci indera spesifik seperti mata atau pendengaran.
*   **Kulit:**
    *   **Penyakit/Dampak:** Kanker kulit.

**2. Sistem Pernapasan:**
*   **Paru-Paru:**
    * 